# Agentes

**Sumario**

1. Introduccion
   1. Que es un agente
   2. Que es una herramienta
   3. Tipos de agentes
<br></br>
2. Agentes de accion
   1. ReAct Zero-shot 
   2. ReAct Conversacional
   3. ReAct Docstore
   4. ReAct Self-ask
<br></br>
3. Agentes de planificacion y ejecucion

In [ ]:
import openai
import os
from langchain.llms import AzureOpenAI

# Lee la clave de API desde el archivo de configuración
with open('config.txt') as f:
    config = dict(line.strip().split('=') for line in f)

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = config.get("OPENAI_API_KEY", "")

# Nombre del despliegue en mi Azure OpenAI Studio is "Davinci003", el modelo es "text-davinci-003"
engine = "Davinci003"
model = "text-davinci-003"
openai_api_version = "2023-12-01" 

# Nombre del despliegue en mi Azure OpenAI Studio para el modelo de embeddings es "TextEmbeddingAda002"
embeddings_engine = "TextEmbeddingAda002"

max_tokens = 1000

llm = AzureOpenAI(
    azure_endpoint=openai.api_base, 
    azure_deployment=engine, 
    openai_api_key=config.get("OPENAI_API_KEY", ""), 
    openai_api_version=openai.api_version,
    # temperature=0, # Podemos poner la temperatura a 0 si queremos reducir la variabilidad de las respuestas
)
llm.openai_api_base = openai.api_base 
llm.max_tokens = max_tokens

# 1 - Introducción

## 1.1 - ¿Qué es un agente?

El concepto de "agente" es una abstracción que permite a nuestro modelo de lenguaje:
* Utilizar calculadoras
* Uyilizar calendarios
* Ejecutar código Python
* Buscar en Internet
* etc.

Este tipo de acciones se realizarán mediante **herramientas**, muchas de las cuales vienen ya dadas por LangChain, y sino siempre podemos diseñar las nuestras propias (como veremos más adelante).

Por lo tanto, un agente está compuesto de dos elementos principales:
* Un LLM base
* Una o multiples herramientas con las que interactuar

Dependiendo de la entrada del usuario, el agente puede decidir cuál, si alguna, de estas herramientas llamar. La utilidad de [**LangChain**](https://python.langchain.com/en/latest/index.html#) se hace evidente en este contexto, ya que proporciona un marco de programación versátil para construir tales sistemas.

<table>
    <tr>
        <td><img src="images_3_3/agent.png" width="800"/></td>
    </tr>
</table>

## 1.2 - ¿Qué es una herramienta?

En Langchain, las herramientas son funciones que pueden ser llamadas por un agente. Se utilizan para realizar tareas específicas, como acceder a datos, generar texto o traducir idiomas.

Las herramientas se definen como objetos `Tool` que contienen tres propiedades:

* `name`: El nombre de la herramienta.
* `description`: Una descripción de la herramienta.
* `function`: La función a ejecutar cuando se llama a la herramienta.

Las herramientas se utilizan para facilitar la creación de agentes complejos que pueden realizar una amplia gama de tareas. Por ejemplo, un agente que ayuda a los usuarios a resolver problemas técnicos podría usar herramientas para acceder a la documentación, buscar soluciones en línea o generar código.

A continuación se presentan algunas herramientas comunes:
* Wikipedia
* DuckDuckGoSearch
* ShellTool

Sin embargo hay una gran cantidad de ellas [en la documentación oficial de LangChain](https://python.langchain.com/docs/integrations/tools) (y su número aumenta cada dia). 

### Wikipedia

Para búsquedas medianamente estructuradas en Wikipedia

`pip install wikipedia`

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

# Para mas informacion acerca de las opciones (e.g), mirar la documentacion:
# https://api.python.langchain.com/en/latest/utilities/langchain.utilities.wikipedia.WikipediaAPIWrapper.html#
wikipedia_eng = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(
    doc_content_chars_max=2000
))

wikipedia_es = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(
    lang="es",
    doc_content_chars_max=2000
))

wikipedia_es.run("Open AI")

### DuckDuckGoSearch

Para busquedas "raw" en internet. 

**Nota:** En este caso no he encontrado manera de indicarle que haga las búsquedas en español, pero quizas hay una manera. Sino, siempre podemos traducir el resultado como paso intermedio a que nuestro modelo lo procese.

`pip install duckduckgo-search`

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.run("Open AI")

### Shell

Permite a los agentes ejecutar comandos de shell. Esto puede ser muy útil para tareas que requieren acceso al sistema operativo, como interactuar con el sistema de archivos o ejecutar aplicaciones.

**Nota:** 
- No funciona en Windows
- Es peligroso dejar el control de nuestro sistema de archivos, lo mejor seria encapsularlo en una maquina virtual y prohibirle ciertos comandos (por si acaso)

In [ ]:
from langchain.tools import ShellTool

shell_tool = ShellTool()

print(shell_tool.run(
    {
        "commands": [
            "echo 'Hello World!'", 
            "ls -lh"
        ]
    }
))

## 1.3 - Tipos de agentes

Hay dos tipos de agentes principales:
* [**Agentes de Acción**](https://python.langchain.com/docs/modules/agents/): estos agentes deciden una acción a tomar y realizan esa acción paso a paso.
* [**Agentes de Planificación y Ejecución**](https://api.python.langchain.com/en/latest/plan_and_execute/langchain_experimental.plan_and_execute.agent_executor.PlanAndExecute.html#) (Experimental): estos agentes primero deciden un plan de acciones a tomar y luego ejecutan esas acciones paso a paso

# 2 - Agentes de Acción

**Pseudocódigo de alto nivel de los agentes de acción:**

1. Se recibe una entrada del usuario.
2. El agente decide qué herramienta utilizar, si es que hay alguna, y cuál debe ser la entrada de esa herramienta.
3. A continuación, se llama a esa herramienta con esa entrada de herramienta y se registra una observación (esto es simplemente la salida de llamar a esa herramienta con esa entrada de herramienta).
4. Ese historial, la entrada de herramienta y observación final se devuelve al agente, y éste decide qué paso dar a continuación.
5. Esto se repite hasta que el agente decide que ya no necesita utilizar ninguna herramienta, y entonces responde directamente al usuario.

**4 tipos de agentes de acción (inspirados del paper ["ReAct: Synergizing Reasoning and Acting in Language Models"](https://arxiv.org/abs/2210.03629)):**
* ReAct Zero-shot 
* ReAct Conversacional
* ReAct Docstore
* ReAct Self-ask

[AgentTypes en la documentacion de LangChain](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html#)

## 2.1 - ReAct Zero-shot

`pip install numexpr`

----

**Nota importante:** Al interactuar con agentes, es muy importante establecer el parámetro `max_iterations` porque los agentes pueden quedarse atrapados en bucles infinitos que consumen muchos tokens. El valor predeterminado es 15 para permitir el uso de muchas herramientas y razonamientos complejos, pero para la mayoría de las aplicaciones, es mejor mantenerlo mucho más bajo.

----

In [ ]:
from langchain.agents import AgentType, Tool, load_tools, initialize_agent

tools = load_tools(
    ["llm-math"], 
    llm=llm
)

zero_shot_agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
zero_shot_agent_executor("Cuanto es 100 + 25?")

Si nos fijamos, el agente utiliza el inglés como idioma "base". Como hemos visto en otras clases, podemos modificar este comportamiento modificando las prompts de la cadena.

In [ ]:
print(zero_shot_agent_executor.agent.llm_chain.prompt.template)

En este caso, vamos a tomar un enfoque diferente y vamos a traducir el texto de español a inglés antes de pasarselo al agente.

In [ ]:
from langchain.callbacks import get_openai_callback
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

def call_agent_with_translate(agent, es_eng_chain, eng_es_chain, query):

    print(f"Input: {query}")

    with get_openai_callback() as cb:
        query_translated = es_eng_chain(query)["text"]
        query_translated = query_translated.replace('\n', '') # Simplemente para mostrarlo mejor en este ejemplo
        es_eng_token_count = cb.total_tokens
    print(f"Input (traducido al inglés): {query_translated}")

    with get_openai_callback() as cb:
        result = agent(query_translated)
        agent_token_count = cb.total_tokens
    print(f"Output: {result['output']}")

    with get_openai_callback() as cb:
        output_translated = eng_es_chain(result["output"])["text"]
        output_translated = output_translated.replace('\n', '') # Simplemente para mostrarlo mejor en este ejemplo
        eng_es_token_count = cb.total_tokens
    print(f"Output (traducido al español): {output_translated}")

    total_tokens = es_eng_token_count + agent_token_count + eng_es_token_count
    print(f'He usado un total de {total_tokens} tokens')

    return result

# Chain para traducir el texto de entrada de español a inglés
translate_eng_es_prompt = ChatPromptTemplate.from_template(
    "Translate the following spanish text to english (write only the translated text): {input}"
)
translate_eng_es_chain = LLMChain(llm=llm, prompt=translate_eng_es_prompt)
# Chain para traducir el texto de salida del modelo de inglés a español
translate_es_eng_prompt = ChatPromptTemplate.from_template(
    # "Traduce el siguiente texto a español, deja las operaciones matematicas sin modificar: {input}"
    "Si el siguiente texto esta en inglés, traducelo al español, en caso contrario, simplemente devuelve el mismo texto: {input}"
)
translate_es_eng_chain = LLMChain(llm=llm, prompt=translate_es_eng_prompt)

In [ ]:
result = call_agent_with_translate(
    zero_shot_agent_executor, 
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Podrias decirme cual el resultado de 100 + 25?"
)

## 2.2 - ReAct Conversacional

El agente zero-shot es interesante, pero **no tiene memoria**. ¿Qué pasa si queremos un asistente que recuerde las cosas de las que hemos hablado y que también pueda razonar sobre ellas y utilizar herramientas? Para eso tenemos el agente ReAct Conversacional:

In [ ]:
from langchain.memory import ConversationBufferMemory

tools = load_tools(
    ["llm-math"], 
    llm=llm
)

memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent_executor = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION ,
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

result = call_agent_with_translate(
    conversational_agent_executor, 
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Podrias decirme cual el resultado de 100 + 25?"
)

In [ ]:
result = call_agent_with_translate(
    conversational_agent_executor, 
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Que es lo último que te he preguntado en nuestra conversación?"
)

## 2.3 - ReAct Docstore

Este tipo de agente es similar a los que hemos visto hasta ahora, pero incluye la interacción con un almacén de documentos. Solo tendrá dos herramientas a su disposición:

* **Search** (Búsqueda). El agente usa esta herramienta para encontrar un artículo relevante.
* **Lookup** (Consulta). El agente intenta encontrar la información correcta en el artículo.

Es el agente por defecto en el caso que queramos hacer **búsquedas simples** en wikipedia o almacenenes de documentos similares.

----

**Nota:** ReAct Docstore no está pensado para trabajar con memoria por defecto. En tal caso tendriamos que preparar nuestro propio Agente customizado

----

In [ ]:
import wikipedia
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

# Podemos cambiar el idiomas de las paginas que recupere el agente 
# (lo cual podria afectar a la calidad y cantidad de información disponible)
#
# Para mas informacion: https://wikipedia.readthedocs.io/en/latest/quickstart.html
wikipedia.set_lang("en")

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

docstore_agent_executor = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.REACT_DOCSTORE, 
    verbose=True,
    max_iterations=10
)

El prompt asociado al modelo difiere de los anteriores en cuanto que contiene varios ejemplos del bucle Pregunta > Pensamiento > Acción > Observación, que incluyen las herramientas de búsqueda y consulta.

**Nota:** Este bucle se explica con más detalle (al igual que con mas ejemplos) en el paper original ["ReAct: Synergizing Reasoning and Acting in Language Models"](https://arxiv.org/abs/2210.03629) 

In [ ]:
print(docstore_agent_executor.agent.llm_chain.prompt.template)

In [ ]:
result = call_agent_with_translate(
    docstore_agent_executor, 
    translate_eng_es_chain,
    translate_es_eng_chain,
    "Cuales han sido las principales investigaciones del filosofo griego Aristoteles?"
)

## 2.4 - ReAct Self-ask

Es nuestra primera elección de agente para cuando queremos utilizar un LLM con el objetivo de extraer información con un motor de búsqueda. El agente hará preguntas de seguimiento y utilizará la funcionalidad de búsqueda para obtener respuestas intermedias que lo ayuden a llegar a una respuesta final.

Utilizaremos el motor de búsqueda DuckDuckGo porque es gratuito y no requiere una clave API.

----

**Nota:** Al igual que ReAct Docstore, ReAct Self-ask no está pensado para trabajar con memoria por defecto. En tal caso tendriamos que preparar nuestro propio Agente customizado

----

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='duckduckgo search'
    )
]

self_ask_with_search_agent_executor = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True, 
    max_iterations=10
)

Como podemos ver a continuación, el prompt del agente es básicamente una serie de ejemplos para mostrarle al LLM cómo hacer preguntas de seguimiento a una herramienta de búsqueda hasta que pueda llegar a la respuesta final.

In [ ]:
print(self_ask_with_search_agent_executor.agent.llm_chain.prompt.template)

In [ ]:
pregunta = "Cual es el nombre de la esposa del actual presidente de los Estados Unidos de America?"

result = call_agent_with_translate(
    self_ask_with_search_agent_executor, 
    translate_eng_es_chain,
    translate_es_eng_chain,
    pregunta
)

# 3 - Agentes de Planificación y Ejecución

Los agentes "Plan-and-Execute" (PaE) logran un objetivo planificando primero qué hacer y luego ejecutando las subtareas. Esta idea se inspira en gran medida en [BabyAGI](https://github.com/yoheinakajima/babyagi) y luego en el documento ["Plan-and-Solve" (Wang et al., 2023)](https://arxiv.org/pdf/2305.04091.pdf).
* La planificación casi siempre la realiza un LLM.
* La ejecución generalmente la realiza un agente de acción (equipado con herramientas).

**Pseudocódigo de alto nivel de los agentes PaE:**
* Se recibe alguna entrada del usuario
* El **planificador** enumera los pasos a seguir
* El **ejecutor** recorre la lista de pasos, ejecutándolos

**Forma típica de construir un agente PaE:**
* **Planificador:** actua a modo de "cerebro" y define que pasos se deben seguir. Se puede invocar mediante la función `load_chat_planner(llm)`.
  * LLM
* **Ejecutor:** decide qué herramientas llamar en cada paso y en qué orden. Se puede invocar con la función `load_agent_executor(llm, tools)`.
  * LLM
  * Kit de herramientas

In [ ]:
from langchain import LLMMathChain

search = DuckDuckGoSearchRun()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    # Esta es la manera "formal" de llamar a 'llm-math'
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [ ]:
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
pae_agent_executor = PlanAndExecute(planner=planner, executor=executor)

In [ ]:
pregunta = "Quién es el actual primer ministro de Gran Bretaña? Cual es su edad elevado a 0.43?"

result = call_agent_with_translate(
    pae_agent_executor, 
    translate_eng_es_chain,
    translate_es_eng_chain,
    pregunta
)

# Siguiente

En la siguiente lección, veremos cómo podemos definir nuestras propias herramientas